In [ ]:
import warnings
import datetime as dt
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['W1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time
    Src['Week Day']                     = Query['datetime'].dt.day_name().map(WEEK_DAY)

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']   = Src['Close'] - Src['Open']
    Src['Variat']   = Src['Close'] / Src['Open'] *100-100
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)

    # ================ Current ================ #
    for x in [0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00]:
        Calc[f'Variat > +{x}%'] = (Calc['Variat'] > +x/100)                 .astype(int)
        Calc[f'Variat < -{x}%'] = (Calc['Variat'] < -x/100)                 .astype(int)


    # ================ Forward ================ #
    Calc[f'Variat +{1}']        =  Calc['Variat'].shift(-1)
    Calc[f'Variat +{1} Pos']    = (Calc[f'Variat +{1}'] > 0)                .astype(int)
    Calc[f'Variat +{1} Neg']    = (Calc[f'Variat +{1}'] < 0)                .astype(int)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [11]:
def STATS_UP(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=1):
        Df = Calc[Calc[f'Variat > +{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat > %':       x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS_UP(Calc)

,Count,Variat > %,Variat +1 Pos,Variat +1 Neg
0,121,5.0,51.2,47.9
1,121,4.5,51.2,47.9
2,122,4.0,50.8,48.4
3,122,3.5,50.8,48.4
4,122,3.0,50.8,48.4
5,122,2.5,50.8,48.4
6,122,2.0,50.8,48.4
7,122,1.5,50.8,48.4
8,123,1.0,50.4,48.8
9,123,0.5,50.4,48.8


In [10]:
def STATS_DOWN(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=0):
        Df = Calc[Calc[f'Variat < -{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat < %':       -x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS_DOWN(Calc)

,Count,Variat < %,Variat +1 Pos,Variat +1 Neg
0,138,-0.5,42.8,56.5
1,138,-1.0,42.8,56.5
2,138,-1.5,42.8,56.5
3,138,-2.0,42.8,56.5
4,138,-2.5,42.8,56.5
5,137,-3.0,43.1,56.2
6,137,-3.5,43.1,56.2
7,137,-4.0,43.1,56.2
8,137,-4.5,43.1,56.2
9,135,-5.0,43.0,56.3


# Snippets

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']   = Src['Close'] - Src['Open']
    Src['Variat']   = Src['Close'] / Src['Open'] *100-100
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    # ================ Current ================ #
    for x in [0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00]:
        Calc[f'Variat > +{x}%'] = (Calc['Variat'] > +x/100)                 .astype(int)
        Calc[f'Variat < -{x}%'] = (Calc['Variat'] < -x/100)                 .astype(int)

    # ================ Forward ================ #
    Calc[f'Variat +{1}']        =  Calc['Variat'].shift(-1)
    Calc[f'Variat +{1} Pos']    = (Calc[f'Variat +{1}'] > 0)                .astype(int)
    Calc[f'Variat +{1} Neg']    = (Calc[f'Variat +{1}'] < 0)                .astype(int)
    return Calc

In [ ]:
def STATS_UP(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=1):
        Df = Calc[Calc[f'Variat > +{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat > %':       x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)


def STATS_DOWN(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=0):
        Df = Calc[Calc[f'Variat < -{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat < %':       -x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)